<a href="https://colab.research.google.com/github/AUT-Student/BigData-HW2/blob/main/BigData_HW2_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><b>In the name of God</b></center>

<b>Course</b>: ‌Big Data
<br>
<b>Description:</b> HomeWork 2 | Question 1
<br>
<b>Developer</b>: Alireza Mazochi (400131075)

# Libraries

In [1]:
import numpy as np 
import time
import random
from collections import deque

# Dataset

In [2]:
!gdown 1PdXgb4w0gtsocKHmeGQ_zPfb3VbW9YhL
!unzip /content/Bigdata_hw2_datasets.zip

Downloading...
From: https://drive.google.com/uc?id=1PdXgb4w0gtsocKHmeGQ_zPfb3VbW9YhL
To: /content/Bigdata_hw2_datasets.zip
100% 6.28M/6.28M [00:00<00:00, 31.0MB/s]
Archive:  /content/Bigdata_hw2_datasets.zip
   creating: Bigdata_hw2_datasets/
   creating: Bigdata_hw2_datasets/q1/
  inflating: Bigdata_hw2_datasets/q1/stream_data_dgim.txt  
   creating: Bigdata_hw2_datasets/q2/
  inflating: Bigdata_hw2_datasets/q2/games.csv  
  inflating: Bigdata_hw2_datasets/q2/ratings.csv  
   creating: Bigdata_hw2_datasets/q3/
  inflating: Bigdata_hw2_datasets/q3/c1.txt  
  inflating: Bigdata_hw2_datasets/q3/c2.txt  
  inflating: Bigdata_hw2_datasets/q3/data.txt  


In [3]:
dataset = [int(x) for x in  open("/content/Bigdata_hw2_datasets/q1/stream_data_dgim.txt").readlines()[0].split("\t")[:-1]]

In [4]:
len(dataset)

40000

# DGIM

In [5]:
class Bucket():
  def __init__(self, size, start, end):
    self.size = size
    self.start = start
    self.end = end

  @staticmethod
  def combine(bucket1, bucket2):
    assert bucket1.size == bucket2.size

    new_size = bucket1.size * 2
    new_start = min(bucket1.start, bucket2.start)
    new_end = max(bucket1.end, bucket2.end)

    return Bucket(size=new_size, start=new_start, end=new_end)
  
  def is_size(self, size):
    return self.size == size

  def is_complete_out_window(self, window_size, counter):
    return self.end + window_size <= counter 

  def is_partial_out_window(self, window_size, counter):
    return self.start + window_size <= counter

  def __str__(self):
    return f"start = {self.start}, end = {self.end}, size = {self.size}"

In [6]:
class DGIM():
  def __init__(self, window_size):
    self.window_size = window_size
    self.dataset = dataset
    self.counter = 0
    self.buckets = deque()

  def read(self, data):
    self.counter += 1

    if data == 0:
      return

    new_bucket = Bucket(size=1, start=self.counter, end=self.counter)
    self.buckets.appendleft(new_bucket)
    
    self._combine_small_buckets()
    self._remove_out_window_bucket()
    
  def _combine_small_buckets(self):
    size = 1
    check = 0

    while(check+2 < len(self.buckets)):

      bucket_0 = self.buckets[check+0]
      bucket_1 = self.buckets[check+1]
      bucket_2 = self.buckets[check+2]

      if bucket_0.is_size(size) and bucket_1.is_size(size) and bucket_2.is_size(size):

        tmp_list = []

        for i in range(check+1):
          tmp_list.append(self.buckets.popleft())

        bucket_1 = self.buckets.popleft()
        bucket_2 = self.buckets.popleft()

        bucket_12 = Bucket.combine(bucket_1, bucket_2)
        self.buckets.appendleft(bucket_12)

        for i in range(check, -1, -1):
          self.buckets.appendleft(tmp_list[i])
        
        check += 1
        size *= 2

      else:
        break

  def _remove_out_window_bucket(self):
    if self.buckets[-1].is_complete_out_window(self.window_size, self.counter):
      self.buckets.pop()

  def predict(self):
    output = 0

    for i in range(len(self.buckets)-1):
      output += self.buckets[i].size

    if self.buckets[-1].is_partial_out_window(self.window_size, self.counter):
      output += 0.5 * self.buckets[-1].size
    else:
      output += self.buckets[-1].size

    return int(output)

  def predict_partial(self, partial_size):
    output = 0

    for i in range(len(self.buckets)):
      if self.buckets[i].is_partial_out_window(partial_size, self.counter):
        output += 0.5 * self.buckets[i].size
        break
      else:
        output += self.buckets[i].size

    return int(output)

  def visualize(self):
    print(f"Window Size = {self.window_size}")
    print(f"Counter = {self.counter}")
    print(f"Buckets = ")
    for i in range(len(self.buckets)):
      print(self.buckets[i])

In [7]:
dgim = DGIM(window_size=1000)

for data in dataset:
  dgim.read(data)

dgim.visualize()

print(f"\nNumber 1 in the last 1000 bits = {dgim.predict()}")
print(f"Number 1 in the last 500 bits = {dgim.predict_partial(500)}")
print(f"Number 1 in the last 200 bits = {dgim.predict_partial(200)}")

Window Size = 1000
Counter = 40000
Buckets = 
start = 39999, end = 39999, size = 1
start = 39995, end = 39995, size = 1
start = 39990, end = 39993, size = 2
start = 39982, end = 39988, size = 4
start = 39970, end = 39981, size = 4
start = 39945, end = 39969, size = 8
start = 39921, end = 39938, size = 8
start = 39885, end = 39920, size = 16
start = 39845, end = 39884, size = 16
start = 39757, end = 39844, size = 32
start = 39669, end = 39756, size = 32
start = 39504, end = 39668, size = 64
start = 39359, end = 39502, size = 64
start = 39025, end = 39354, size = 128
start = 38391, end = 39024, size = 256

Number 1 in the last 1000 bits = 508
Number 1 in the last 500 bits = 220
Number 1 in the last 200 bits = 76


# ExactCounter

In [14]:
class ExactCounter():
  def __init__(self, window_size):
    self.window_size = window_size
    self.window = deque()

  def read(self, data):
    if len(self.window) == self.window_size:
      self.window.popleft()

    self.window.append(data)
  
  def predict(self):
    output = 0
    for i in range(len(self.window)):
      if self.window[i] == 1:
        output += 1

    return output

  def predict_partial(self, partial_size):
    output = 0
    for i in range(min(partial_size, len(self.window))):
      if self.window[len(self.window)-1-i] == 1:
        output += 1
    
    return output

In [15]:
exact_counter = ExactCounter(window_size=1000)

for data in dataset:
  exact_counter.read(data)

print(f"Number 1 in the last 1000 bits = {exact_counter.predict()}")
print(f"Number 1 in the last 500 bits = {exact_counter.predict_partial(500)}")
print(f"Number 1 in the last 200 bits = {exact_counter.predict_partial(200)}")

Number 1 in the last 1000 bits = 391
Number 1 in the last 500 bits = 190
Number 1 in the last 200 bits = 77


# Analysis

## Main Dataset

In [ ]:
dgim = DGIM(window_size=1000)

start_time = time.time()
for data in dataset:
  dgim.read(data)
end_time = time.time()

dgim_read_time = end_time - start_time

start_time = time.time()
dgim_predict = dgim.predict()
end_time = time.time()

dgim_predict_time = end_time - start_time

exact_counter = ExactCounter(window_size=1000)

start_time = time.time()
for data in dataset:
  exact_counter.read(data)
end_time = time.time()

exact_read_time = end_time - start_time

start_time = time.time()
exact_count = exact_counter.predict()
end_time = time.time()

exact_count_time = end_time - start_time

In [ ]:
print(f"Dgim Reading Time = {round(dgim_read_time*1000*1000/len(dataset), 3)} us | ExactCounter Reading Time = {round(exact_read_time*1000*1000/len(dataset), 3)} us")
print(f"Dgim Prediction Time = {round(dgim_predict_time*1000, 3)} ms | ExactCounter Counter Time = {round(exact_count_time*1000, 3)} ms")
print(f"Dgim Prediction = {dgim_predict} | Exact Count = {exact_count}")

Dgim Reading Time = 2.803 us | ExactCounter Reading Time = 0.477 us
Dgim Prediction Time = 0.046 ms | ExactCounter Counter Time = 0.156 ms
Dgim Prediction = 508 | Exact Count = 391


## Auxiliary Datasets

In [ ]:
dataset_normal = list(np.random.choice([0, 1], size=(1000*1000, ) , p=[1/2, 1/2]))
dataset_more_one = list(np.random.choice([0, 1], size=(1000*1000, ) , p=[1/4, 3/4]))
dataset_more_zero = list(np.random.choice([0, 1], size=(1000*1000, ) , p=[3/4, 1/4]))

auxiliary_datasets = [dataset_normal, dataset_more_one, dataset_more_zero]

In [ ]:
dataset_names = ["Normal", "More One", "More Zero"]
for i, auxiliary_dataset in enumerate(auxiliary_datasets):

  dgim = DGIM(window_size=100*1000)

  start_time = time.time()
  for data in auxiliary_dataset:
    dgim.read(data)
  end_time = time.time()

  dgim_read_time = end_time - start_time

  start_time = time.time()
  dgim_predict = dgim.predict()
  end_time = time.time()

  dgim_predict_time = end_time - start_time

  exact_counter = ExactCounter(window_size=100*1000)

  start_time = time.time()
  for data in auxiliary_dataset:
    exact_counter.read(data)
  end_time = time.time()

  exact_read_time = end_time - start_time

  start_time = time.time()
  exact_count = exact_counter.predict()
  end_time = time.time()

  exact_count_time = end_time - start_time

  print(f"Dataset = {dataset_names[i]}")
  print(f"Dgim Reading Time = {round(dgim_read_time*1000*1000/len(auxiliary_dataset), 3)} us | ExactCounter Reading Time = {round(exact_read_time*1000*1000/len(auxiliary_dataset), 3)} us")
  print(f"Dgim Prediction Time = {round(dgim_predict_time*1000, 3)} ms | ExactCounter Counter Time = {round(exact_count_time*1000, 3)} ms")
  print(f"Dgim Prediction = {dgim_predict} | Exact Count = {exact_count}")
  print("==========")

Dataset = Normal
Dgim Reading Time = 3.943 us | ExactCounter Reading Time = 0.448 us
Dgim Prediction Time = 0.021 ms | ExactCounter Counter Time = 227.001 ms
Dgim Prediction = 57265 | Exact Count = 50253
Dataset = More One
Dgim Reading Time = 5.279 us | ExactCounter Reading Time = 0.443 us
Dgim Prediction Time = 0.022 ms | ExactCounter Counter Time = 233.156 ms
Dgim Prediction = 78706 | Exact Count = 75069
Dataset = More Zero
Dgim Reading Time = 2.028 us | ExactCounter Reading Time = 0.458 us
Dgim Prediction Time = 0.019 ms | ExactCounter Counter Time = 233.382 ms
Dgim Prediction = 29038 | Exact Count = 24653
